In [ ]:
import pandas as pd
import numpy as np
import os
import sys

from sklearn.datasets import make_classification

from sklearn.datasets import load_breast_cancer

sys.path.append(os.path.join(os.pardir))
from functions import *
from model_tuner import *

In [ ]:
bc = load_breast_cancer(as_frame=True)["frame"]

In [ ]:
bc_cols = [cols for cols in bc.columns if "target" not in cols]

In [ ]:
X = bc[bc_cols]

In [ ]:
y = bc["target"]

In [ ]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression(class_weight="balanced", max_iter=1000)

estimator_name = "lg"
# Set the parameters by cross-validation
tuned_parameters = [{estimator_name + "__C": np.logspace(-4, 0, 3)}]

In [ ]:
kfold = True
calibrate = False

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = Model(
    name="Logistic Regression",
    estimator_name=estimator_name,
    calibrate=calibrate,
    estimator=lr,
    kfold=kfold,
    stratify_y=True,
    grid=tuned_parameters,
    randomized_grid=True,
    n_iter=3,
    scoring=["roc_auc"],
    n_splits=10,
    n_jobs=-2,
    random_state=42,
)


model.grid_search_param_tuning(X_train, y_train, f1_beta_tune=True)

model.kfold = False

y_prob = model.predict_proba(X_test)

### F1 Weighted
y_pred = model.predict(X_test, optimal_threshold=True)

In [ ]:
y_pred

In [ ]:
y_prob